In [1]:
print("Hello World")

Hello World


In [ ]:
pip install -r requirements.txt


# Visualize Dataset

In [6]:
import pandas as pd 

df_ = pd.read_csv("../data/Air Quality Ho Chi Minh City.csv")
df = df_.copy()
df.head()

,date,Station_No,TSP,PM2.5,O3,CO,NO2,SO2,Temperature,Humidity
0,23-02-2021 21:00,1,32.935714,15.604762,55.431381,1330.451429,112.740762,393.000000,28.361905,63.188095
1,23-02-2021 22:00,1,30.932353,14.594118,58.197176,1200.603529,112.366471,377.588235,28.320588,63.773529
2,23-02-2021 23:00,1,27.645000,13.436667,55.029433,1177.897000,112.700433,372.476667,28.336667,64.205000
3,24-02-2021 00:00,1,24.380000,12.365000,54.767700,1267.476000,112.480867,389.070000,28.305000,64.735000
4,24-02-2021 01:00,1,22.521667,11.636667,53.786200,1322.293000,114.331500,393.000000,28.300000,65.188333


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52548 entries, 0 to 52547
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         52548 non-null  object 
 1   Station_No   52548 non-null  int64  
 2   TSP          52488 non-null  float64
 3   PM2.5        52548 non-null  float64
 4   O3           41938 non-null  float64
 5   CO           43483 non-null  float64
 6   NO2          46882 non-null  float64
 7   SO2          41542 non-null  float64
 8   Temperature  48111 non-null  float64
 9   Humidity     48116 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 4.0+ MB


Tách date thành => 3 cột Ngày Tháng NămNăm

In [ ]:
print(df.isnull().sum(axis=0)) #checknull cot

date               0
Station_No         0
TSP               60
PM2.5              0
O3             10610
CO              9065
NO2             5666
SO2            11006
Temperature     4437
Humidity        4432
dtype: int64
